<a href="https://colab.research.google.com/github/cy643/generative_ai/blob/main/0401_hw6/0401_hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0401_hw6 我的對話機器人
#### 主題一 : 延續上週作業，參考老師的範例，更改成可以持續對話的版本

[Ollama](https://ollama.com/) 可以讓我們在自己的機器上跑開源的大型語言模型, 並且用 API 的方式呼叫。這裡我們介紹在 Colab 上跑, 並且分別用 OpenAI 的 API, 及 [`aisuite` 套件](https://github.com/andrewyng/aisuite) 來使用 Ollama 提供的大型語言模型。

### 1. 安裝並執行 Ollama

首先是到官網抓下安裝程式, 並且安裝。

In [14]:
## 官網抓下安裝程式
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [33]:
## API 呼叫
!nohup ollama serve &
## 把會用到的模型抓下來
!ollama pull gemma3:1b

nohup: appending output to 'nohup.out'
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 7cd4618c1faf... 100% ▕▏ 815 MB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling dd084c7d92a3... 100% ▕▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕▏   77 B                         
pulling 120007c81bf8... 100% ▕▏  492 B                         
verifying sha256 digest 
writing manifest 
success 


### 2. OpenAI API 使用

因為 ChatGPT 大概是最早紅的大型語言模型, 因此許多大型語言模型, 都和 OpenAI API 相容, Ollama 也不例外。

In [28]:
import openai
from openai import OpenAI
## 本次作業無需使用金鑰，自定義即可
api_key = "ollama"
## 如同一般 OpenAI API 打開 client 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 port 是 11434。
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 3. 打造對話機器人 web app!

In [17]:
!pip install gradio

import gradio as gr

對話機器人 app 設定

In [29]:
title = "BotTender"
description = "告訴我你現在的心情、偏好的口味，或是你冰箱裡剩下些什麼——我會為你調出一杯剛剛好的酒。Cheers 🥂"
system = "你是一位虛擬的調酒師 AI，低調紳士、輕熟穩重，擁有豐富的調酒知識與絕佳的感受力。像一位陪使用者放鬆的好夥伴，但是請根據台灣人的習慣回復，並限制在一百字內。"
description = "歡迎光臨，我是BotTender，一位用味道說話的虛擬調酒師。現在的你，是想讓自己放鬆，還是想找回一點熱情？"
model = "gemma3:1b"

initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [31]:
state = gr.State(initial_messages)

def botTender(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]
    return messages, messages

chatbot = gr.Chatbot(type="messages")

In [34]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🥂 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=botTender, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4fe7379e5540b0c96e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4fe7379e5540b0c96e.gradio.live
